The dataset is used for this competition is synthetic, but based on a real dataset and generated using a CTGAN. The original dataset deals with predicting the biological response of molecules given various chemical properties. Although the features are anonymized, they have properties relating to real-world features.

Import

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

Load

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Read

In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-oct-2021/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-oct-2021/test.csv")
submission = pd.read_csv("/kaggle/input/tabular-playground-series-oct-2021/sample_submission.csv")

In [ ]:
train

In [ ]:
test

In [ ]:
submission

Shorten train

In [ ]:
short_train = train[: 250000]
short_train

Analyse

In [ ]:
import seaborn as sns

sns.histplot(short_train.target)

Combine

In [ ]:
target = short_train.target

combi = short_train.drop(['target'], axis=1).append(test)
combi

Analyse combi

In [ ]:
combi.info()

Check for null values

In [ ]:
combi.isnull().sum().sum()

Standardise

In [ ]:
#combi = (combi - combi.mean()) / np.std(combi)
#combi

Normalise

In [ ]:
combi = (combi - combi.min()) / (combi.max() - combi.min())
combi

Define X and y

In [ ]:
y = target
X = combi[: len(short_train)]
X_test = combi[len(short_train):]
X.shape, X_test.shape, y.shape

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2

selector = SelectKBest(chi2, k=60)

X = selector.fit_transform(X, y)
X_test = selector.transform(X_test)
y.shape, X.shape, X_test.shape

Split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=1, shuffle=True)
X_train.shape, X_val.shape, y_train.shape,y_val.shape, X_test.shape

Convert numpy arrays to tensor arrays

In [ ]:
import numpy as np
import torch

X_train = X_train.astype(np.float32)
X_val = X_val.astype(np.float32)
X_test = X_test.astype(np.float32)

X_train=torch.from_numpy(X_train)
X_val=torch.from_numpy(X_val)
X_test = torch.from_numpy(X_test)


y_train = y_train.to_numpy()
y_val = y_val.to_numpy()

y_train = y_train.astype(np.float32)
y_val = y_val.astype(np.float32)

y_train=torch.from_numpy(y_train)
y_val=torch.from_numpy(y_val)

In [ ]:
y_train=y_train.view(y_train.shape[0],1)
y_val=y_val.view(y_val.shape[0],1)

Select Model

In [ ]:
class Logistic_Reg_model(torch.nn.Module):
 def __init__(self,no_input_features):
   super(Logistic_Reg_model,self).__init__()
   self.layer1=torch.nn.Linear(no_input_features,30)
   self.layer2=torch.nn.Linear(30,1)
 def forward(self,x):
   y_predicted=self.layer1(x)
   y_predicted=torch.sigmoid(self.layer2(y_predicted))
   return y_predicted

In [ ]:
model=Logistic_Reg_model(X_train.shape[1])

Optimiser

In [ ]:
criterion=torch.nn.BCELoss()
optimizer=torch.optim.SGD(model.parameters(),lr=0.01)

Train data

In [ ]:
number_of_epochs=10000
for epoch in range(number_of_epochs):
 y_prediction=model(X_train)
 loss=criterion(y_prediction,y_train)
 loss.backward()
 optimizer.step()
 optimizer.zero_grad()
 if (epoch+1)%100 == 0:
   print('epoch:', epoch+1,',loss=',loss.item())

Predict on data

In [ ]:
with torch.no_grad():
 y_pred=model(X_val)
 y_pred_class=y_pred.round()
 accuracy=(y_pred_class.eq(y_val).sum())/float(y_val.shape[0])
 print(accuracy.item())

In [ ]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_val, y_pred.round()))

Predict on test set

In [ ]:
with torch.no_grad():
 prediction=model(X_test)
 prediction_class=prediction.round()

In [ ]:
prediction = prediction.round()
#prediction[prediction < 0] = 0
prediction.shape

In [ ]:
submission.target = prediction.detach().numpy().astype(int)
submission

In [ ]:
submission.to_csv('submission.csv', index=False)
submission = pd.read_csv("submission.csv")
submission